### Install PyMySQL

In [ ]:
!pip install pymysql

### Install python-dotenv

In [ ]:
!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

### Import PyMySQL

In [ ]:
import pymysql

### MySQL Configuration

In [ ]:
class config:
  HOST = os.getenv('MYSQL_HOST')
  PORT = int(os.getenv('MYSQL_PORT'))
  USER = os.getenv('MYSQL_USER')
  PASSWORD = os.getenv('MYSQL_PASSWORD')
  DB = os.getenv('MYSQL_DB')
  CHARSET = os.getenv('MYSQL_CHARSET')


### Connect DB

In [ ]:
connection = pymysql.connect(host=config.HOST,
                             port=config.PORT,
                             user=config.USER,
                             password=config.PASSWORD,
                             db=config.DB,
                             charset=config.CHARSET)

### Show tables in DB

In [ ]:
with connection.cursor() as cursor:
  cursor.execute("show tables;")
  tables = cursor.fetchall()
tables


### Import Pandas

In [ ]:
import pandas as pd

### Select data from audible_data

In [ ]:
sql = 'SELECT * FROM audible_data '
audible_data = pd.read_sql(sql,connection)
audible_data = audible_data.set_index('Book_ID')
audible_data

### Select data from audible_transaction

In [ ]:
sql = 'SELECT * FROM audible_transaction '
audible_transaction = pd.read_sql(sql,connection)
audible_transaction.index += 1
audible_transaction

### Join audible_transaction and audible_data

In [ ]:
transaction = audible_transaction.merge(audible_data, how='left', left_on='book_id', right_on='Book_ID')
transaction["Price"] = transaction.apply(lambda x : x['Price'].replace('$',''),axis=1)
transaction

### Import Requests

In [ ]:
import requests

### Request Coversion rate API

In [ ]:
url = 'https://r2de2-workshop-vmftiryt6q-ts.a.run.app/usd_thb_conversion_rate'
r = requests.get(url)
conversion_rate = r.json()
conversion_rate = pd.DataFrame(conversion_rate)
conversion_rate = conversion_rate.reset_index().rename(columns = {'index':'date'})
conversion_rate['date'] = pd.to_datetime(conversion_rate['date']).dt.date
conversion_rate

In [ ]:
transaction['date'] = transaction['timestamp']
transaction['date'] = pd.to_datetime(transaction['date']).dt.date
transaction['Price'] = transaction['Price'].astype(float)
transaction

### Join Transaction and Conversion_rate

In [ ]:
final_df = transaction.merge(conversion_rate, how="left", left_on="date", right_on="date")
final_df['THB'] = final_df['Price']*final_df['conversion_rate']
final_df.drop('date',axis=1)

### Create CSV

In [95]:
final_df.to_csv('output.csv', index= False)